# 1. Hints on using Jupyter notebooks  

A document that is conveneint for instruction because it allows to combine in one file both descriptive text about concepts being taught (maths, figures), and code which can be executed, modified and rerun to examin the effects of those changes. 

## 1.1 Contents of a notebook
The notebook consistes of a sequence of cells. Each is one of two cell types:
* Markdown: containing descriptive text
* Code: with or without the results from the execution of that cell's code.

## 1.2 Notebook graphical interface 

Like any other document interface it takes some getting used to. 

* Top line indicates the name of the notebook file you are viewing, editing and running;  the language being interpretted. 
* Second line provides 
  * document toolbar for familiar commands such as opening a new notebook, saving the changes in the current open notebook. 
  * Edit-ing the existing notebook cells,
  * Insert-ing new cells
  * Cell commands for running code in a cell and changing the cell type between code and markdown.
  * A notebook is connected to an kernel which contains the state of the variables and functions defined in your notebook. This kernel also runs the code in a cell(s) when you tell it to.
  * Help  menu if you get lost.
* Second line also shows:
  * Whether you are in edit mode (pencil icon) or command mode (no pencil visible)
  * Name and version of language interpreter running in your kernel 
* Third row contains button that are shortcuts to commonly used commands from the menu and keyboard.
  * To run a cell's code, press run button (or shift-enter)
  * __Importantly if you want to stop the execution of a cell's code, press the stop botton (square box).__
  * When code is still running, you will see an hourglass in your browser's tab, and the icon next to the interpreter will be a solid gray disk (running) rather than a circle (not running). 

* The **main content area**
  * Large rectangular region below button bar. Contains the cells: additional rectangular regions.  At any one time only once cell is selected. This is indicated by a green bar (edit mode) or blue bar (command mode).
* Scroll bars:
  * Vertical scroll bars occur in two places throughout the notebook. On the extreme right is the main vertical scroll bar  to move through all the cells within the notebook.
  * Within each cell, there can be an additional mini vertical scroll bar which allows moving through the contents of the cell. 

## 1.3 Keyboard interface
The most important keyboard shortcuts (cf. the "Help" menu) are
* **cursor keys** to select cells (up and down)
* Switching between nodes:
  * **Enter** to go from command mode to edit mode (for changing cell contents)
  * (**Esc** would go back to command mode.)
  
* **Shift+Enter** to *execute and advance* a cell
  * While experimenting with different values in the same cell, **Ctrl+Enter** is also handy, which executes but does not advance the cursor.

## 1.4 Summary of edit mode short cuts (see also Help menu):
<img src="./text_figures/JupyterKeyboardShortCuts_EditMode.png" alt="Drawing" style="width: 500px;"/>

## 1.5 Executing cells
Cells can be executed in any order. Select one with the up/down cursor keys, and press shift+enter. 

The `In [ ]` will change to `In [3]` if it is the 3rd cell run

`In [*]` means the cell is being executed

# 2. VAEs and GANs


## 2.1 Overview of the codes provided for this course 
The [vaegan](/vaegan) package contains several modules with the custom classes needed to build and train the VAEs,  GANs, and VAEGAN models. 


### 2.1.1 GAN model
[vaegan.gan](/vaegan/gan.py) contains subclasses of the Keras Model class. These implement the generator and discriminator submodles of the GAN model. This file also implements an extension of GAN known as a conditionial auxillary classifier GAN which can classify the images that it generates. 

### 2.1.2 VAE model
[vaegan.vae](/vaegan/vae.py) contains subclasses of the Keras Model class. These implement the encoder and decoder submodles of the VAE model. This filel also implements an extension of VAE known as a conditional VAE which can be which learns to densities of each class label, and the decoder can be used to generate new images for specific classes you desire. Hence steerable image generation. 

### 2.1.3 VAEGAN
You will be implementing a VAEGAN model that combines the strengths of VAEs and GANs. You model should contains subclasses of the Keras Model class that implement the encoder, decoder, and discriminator submodels of the VAE-GAN. Each submodel contains:
1. A constructor method (`__init__`) that creates a set of layers (e.g. convolutional, dense, and batch normalization)
2. A `call` method that performs a forward pass (sending the data through each layer) and returns the submodel output
3. `get_config` and `from_config` methods which are used by Keras to store and retrieve parameters when models are saved and loaded. 

These are the submodels:
* Encoder: This submodel compresses the input 2D image into a compact vector representation. We'll call this the latent representation. Since this is a _variational_ autoencoder, the latent representations from the encoder are actually _probabilistic_ in nature. The Encoder outputs the mean and the log-transformed variance of the latent representation. 

* Decoder: This does the opposite of the Encoder, reconstructing the 2D image from its latent representation. It uses transposed convolution layers to "reverse" the convolution operations performed in the Encoder. The Decoder also serves as the generator in the GAN portion of the model. By feeding it random latent vectors, we can magically synthesize new images. 

* Discriminator: This is the key to the GAN portion of the model. The Discriminator classifies images as real or fake, where fake refers to any image that comes out of the Decoder (reconstructions and synthetic images). By training the Decoder in competition with this Discriminator, the Decoder learns to produce more realistic images that can trick the Discriminator. 

Finally, there is a VAEGAN class that integrates the submodels and defines the training procedure. It has these important methods:
1. The constructor method (`__init__`) instantiates the submodels and stores them as object attributes. It also sets up the needed loss functions.
2. The `train_step` method defines what happens in a single training iteration (i.e. one mini-batch). This includes the forward pass through each submodel, computation of losses, backpropagation, and weight updates. 

### 2.1.4 Conditional VAE- conditional Auxillary Classifier GAN
An extension to the VAE-GAN architecture is the conditional VAE-condtitional Auxillary Classifier GAN, which combines the conditional VAE (see above VAE model section) and the conditional aux classifier GAN (see above GAN model section) and thereby inherits the strengths and capabilities of those two advanced architectures. This new model receives a class label as an additional input. This allow images to be synthesized for specified classes, rather than purely at random. E.g. instead of generating a random digit image, we can tell the conditional Decoder to generate a "4". The model classes are implemented in [vaegan.conditional.models](/vaegan/conditional.models.py). These include a conditional Encoder and Decoder which take the class label as an additional input and a multi-task Discriminator. This modified Discriminator performs two prediction tasks: 1) classify real vs. fake images and 2) predict the image class label. The Decoder, trained in competition with the Discriminator, now needs to learn to produce images that look realistic (tricking the Discriminator on task 1) _and_ strongly resemble the given class label (the Discriminator classifies them correctly on task 2).

### 2.1.5 Callbacks within the package
[vaegan.callbacks](/vaegan/callbacks.py) contains custom callbacks, which are functions called at various points during model training.

* SaveImages saves example reconstructions and synthetic images after each epoch.

* SaveModel saves model checkpoints after every given number of epochs

## 2.2 Accessing the datasets for this course
[vaegan.data](/vaegan/data.py) contains classes to load and preprocess each dataset. See applications below for the descriptions of the datasets. 

 

## 2.3 Applications and data for this course
For each of the following applications, there is a train*.py and test*.py script. The training script trains the VAE-GAN (implemented in vaegan.models) and saves the trained model, as well as examples of model output images at every epoch. The test script should be run afterwards. It will make sure the model can be loaded, use the encoder to compress test images, plot the latent representations by class, and create some figures of the reconstructed and synthetic images. 

The image_viewer.ipynb contains an interactive tool to view the model output images and compare results throughout training. 

### 2.3.1 MNIST handwritten digits

The Modified National Institute of Standards and Technology (MNIST) digit dataset is a classic machine learning and computer vision dataset. It contains grayscale images of handwritten digits 0-9. The training set contains 60,000 images while the test set contains 10,000 images. 

![mnist](/images/mnist_examples.png)


### 2.3.2 Alzheimer's Disease brain images

Structural magnetic resonance images (MRIs) for about 2000 subjects were sourced from the ADNI study. The subjects either have Alzheimer's Disease or are cognitively normal. We have done some light preprocessing on these images, including removing the skull and normalizing pixel intensities. Though the images are originally 3D, we have selected 2D coronal (parallel to the face) slices through the middle of the right hippocampus. The [hippocampus](https://www.researchgate.net/profile/Emilie-Gerardin/publication/278643309/figure/fig11/AS:788483291492355@1565000474880/Coronal-sagittal-and-axial-MRI-views-of-the-hippocampus-A-3D-structure-is-superposed.ppm) and the surrounding medial temporal lobe are typically the most strongly affected brain regions in Alzheimer's Disease. Note the hippocampal atrophy in the examples below. 

![brains](/images/ad_examples.png)